### Retrieving data

***Result:*** 
The chosen [data](https://s5phub.copernicus.eu/dhus/odata/v1/Products('93ea5d16-84dc-4d42-bd77-9fa7120376e4')/$value) was narrowed down with search query 

***TODO:*** 
Determine the appropriate keyword arguments necessary for querying based on user input

In [34]:
# connect to the API
from sentinelsat import SentinelAPI, read_geojson, geojson_to_wkt
from datetime import date

"""
username & pwd: "s5pguest"
api_url = "https://s5phub.copernicus.eu/dhus"

show_progressbars & timeout is optional
"""
api = SentinelAPI("s5pguest", "s5pguest", "https://s5phub.copernicus.eu/dhus", show_progressbars=True,timeout=60)

# Search product ID using OpenSearch
data_date = date=(date(2023, 5, 7), date(2023, 5, 9))
product_type = "L2__CH4___"
kw = {
    'orbitnumber': 28826
}

result = api.query(date=data_date, producttype=product_type, **kw) # important: producttype='L2__CH4___'

for line in result:
    print (line)

d5301239-5c22-4c97-a06d-bbac865fb092


---
### Retrieving data (2nd round)

Necessary keyword arguments: {}

***Result:*** 
The chosen [data](https://s5phub.copernicus.eu/dhus/odata/v1/Products('93ea5d16-84dc-4d42-bd77-9fa7120376e4')/$value) was narrowed down with search query 

---
## Following beginnner's guide for sentinel sentinel hub

### **NOTE: this code won't be used as copernicus open data hub and sentinelsat python library will be used as per client's suggestion**

In [ ]:
from oauthlib.oauth2 import BackendApplicationClient
from requests_oauthlib import OAuth2Session
from PIL import Image
import io
import numpy as np
import matplotlib.pyplot as plt

CLIENT_ID = "3a8e596f-4e0d-47b5-8497-0f19cebc482f"
CLIENT_SECRET = "Fr2zIcdT%!Agy!?>j&K+#CZz,V]iq^Mj*sAyaQnE"

# set up credentials
client = BackendApplicationClient(client_id=CLIENT_ID)
oauth = OAuth2Session(client=client)

# get an authentication token
token = oauth.fetch_token(token_url='https://services.sentinel-hub.com/oauth/token',
                          client_id=CLIENT_ID, client_secret=CLIENT_SECRET)

bbox = [-87.72171, 17.11848, -87.342682, 17.481674]
start_date = "2020-06-01"
end_date = "2020-08-31"
collection_id = "sentinel-2-l2a"

evalscript = """
//VERSION=3
function setup() {
  return {
    input: ["B02", "B03", "B04"],
    output: { 
      bands: 3, 
      sampleType: "AUTO" // default value - scales the output values from [0,1] to [0,255].
    }
  }
}

function evaluatePixel(sample) {
  return [2.5 * sample.B04, 2.5 * sample.B03, 2.5 * sample.B02]
}
"""

json_request = {
    'input': {
        'bounds': {
            'bbox': bbox,
            'properties': {
                'crs': 'http://www.opengis.net/def/crs/OGC/1.3/CRS84'
            }
        },
        'data': [
            {
                'type': 'S2L2A',
                'dataFilter': {
                    'timeRange': {
                        'from': f'{start_date}T00:00:00Z',
                        'to': f'{end_date}T23:59:59Z'
                    },
                    'mosaickingOrder': 'leastCC',
                },
            }
        ]
    },
    'output': {
        'width': 1024,
        'height': 1024,
        'responses': [
            {
                'identifier': 'default',
                'format': {
                    'type': 'image/jpeg',
                }
            }
        ]
    },
"evalscript": evalscript
}

# Set the request url and headers
url_request = 'https://services.sentinel-hub.com/api/v1/process'
headers_request = {
    "Authorization" : "Bearer %s" %token['access_token']
}

#Send the request
response = oauth.request(
    "POST", url_request, headers=headers_request, json = json_request
)

# read the image as numpy array
image_arr = np.array(Image.open(io.BytesIO(response.content)))

# plot the image for visualization
plt.figure(figsize=(16,16))
plt.axis('off')
plt.tight_layout()
plt.imshow(image_arr)

ModuleNotFoundError: No module named 'oauthlib'